In [2]:
:dep itertools
:dep ndarray
:dep ndarray-interp
:dep vawt = {path="../"}
:dep csv
:dep ndarray-csv
:dep plotters = {default_features = false, features = ["evcxr", "all_series"]}
:dep log
:dep env_logger

use std::fs::File;
use std::f64::consts::{FRAC_PI_2, PI};
use std::error::Error;

use itertools::Itertools;
use csv::ReaderBuilder;
use ndarray::{array, Array2, Axis, Array, Array1};
use ndarray_csv::Array2Reader;
use plotters::prelude::*;
use log::{LevelFilter, info, warn};
use env_logger::Env;

use vawt::areofoil::Aerofoil;
use vawt::{VAWTSolver, VAWTSolution};

In [3]:
env_logger::init();
log::set_max_level(LevelFilter::Info);

In [4]:
fn read_array(path: &str) -> Result<Array2<f64>, Box<dyn Error>> {
    let file = File::open(path)?;
    let mut reader = ReaderBuilder::new()
        .has_headers(false)
        .trim(csv::Trim::All)
        .delimiter(b',')
        .from_reader(file);

    let mut arr: Array2<f64> = reader.deserialize_array2_dynamic().unwrap();
    for mut datapoint in arr.axis_iter_mut(Axis(0)) {
        datapoint[0] = datapoint[0].to_radians();
    }
    Ok(arr)
}

In [5]:
let aerofoil = Aerofoil::builder()
    .add_data_row(read_array("NACA0018/NACA0018Re0080.data")?, 80_000.0)?
    .add_data_row(read_array("NACA0018/NACA0018Re0040.data")?, 40_000.0)?
    .add_data_row(read_array("NACA0018/NACA0018Re0160.data")?, 160_000.0)?
    .set_aspect_ratio(12.8)
    .update_aspect_ratio(true)
    .symmetric(true)
    .build()?;
let aerofoil: &'static Aerofoil = Box::leak(Box::new(aerofoil));
let n = 90;
let mut solver = VAWTSolver::new(&aerofoil);
solver.re(31_300.0)
    .solidity(0.3525)
    .n_streamtubes(n)
    .epsilon(0.0001);

In [13]:
solver.tsr(2.8)
    .particles(8)
    .iterations(50);
let solution = solver.solve_optimize_beta();

let d_t_half = PI / n as f64;
let theta = Array::linspace(d_t_half, 2.0 * PI - d_t_half, n);

evcxr_figure((640 * 2 ,480), |root| {    
    let (left, right) = root.split_horizontally(640);

    left.titled("induction factor and c_tan", ("Arial", 20).into_font())?;
    // Create a chart context with the specified ranges
    let mut chart_left = ChartBuilder::on(&left)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..360.0, -1f64..1f64)?;

    // Customize the x-axis labels
    chart_left.configure_mesh()
        .x_labels(10)
        .y_labels(10)
        .x_label_formatter(&|x| format!("{:.1}°", x))
        .draw()?;

    chart_left.draw_series(LineSeries::new(
        theta.iter().map( |&theta| (theta.to_degrees(), solution.a(theta))), 
        &RED
    ))?;
    chart_left.draw_series(LineSeries::new(
        theta.iter().map( |&theta| (theta.to_degrees(), solution.c_tan(theta))), 
        &BLUE
    ))?;

    right.titled("alpha and beta", ("Arial", 20).into_font())?;
    // Create a chart context with the specified ranges
    let mut chart_right = ChartBuilder::on(&right)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..360.0, -20f64..20f64)?;

    // Customize the x-axis labels
    chart_right.configure_mesh()
        .x_labels(10)
        .y_labels(10)
        .x_label_formatter(&|x| format!("{:.1}°", x))
        .y_label_formatter(&|y| format!("{:.1}°", y))
        .draw()?;

    chart_right.draw_series(LineSeries::new(
        theta.iter().map(|&theta| (theta.to_degrees(), solution.beta(theta).to_degrees())),
        &GREEN
        ))?
        .label("Pitch angle beta")
        .legend(|(x, y)| PathElement::new(vec![(x, y), (x + 20, y)], &GREEN));

    chart_right.draw_series(LineSeries::new(
        theta.iter().map(|&theta| (theta.to_degrees(), solution.alpha(theta).to_degrees())),
        &BLUE
        ))?
        .label("Angle of Attack")
        .legend(|(x, y)| PathElement::new(vec![(x, y), (x + 20, y)], &BLUE));
    chart_right.configure_series_labels()
        .border_style(&BLACK)
        .background_style(&WHITE.mix(0.8))
        .draw()?;
    Ok(())
})


induction factor and c_tan
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
-1.0
 
 
 
-0.5
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
 
0.0°
 
 
 
50.0°
 
 
 
100.0°
 
 
 
150.0°
 
 
 
200.0°
 
 
 
250.0°
 
 
 
300.0°
 
 
 
350.0°
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="43,143 49,180 56,178 63,182 69,184 76,185 83,183 89,183 96,183 103,183 109,183 116,183 123,184 129,185 136,185 143,185 149,185 156,184 163,185 169,184 176,184 183,185 189,186 196,187 203,187 209,189 216,189 223,190 229,189 236,191 242,191 249,192 256,193 262,195 269,196 276,197 282,198 289,199 296,200 302,201 309,202 316,204 322,206 329,211 336,226 342,223 349,209 356,200 362,194 369,191 376,188 382,185 389,181 396,177 402,173 409,169 416,164 422,159 429,158 436,157 442,155 449,154 455,154 462,153 469,154 475,153 482,152 489,153 495,152 502,152 509,152 515,153 522,153 529,152 535,152 542,152 549,151 555,150 562,150 569,152 575,151 582,152 589,152 595,154 602,155 609,157 615,149 622,149 629,135 635,0 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="43,223 49,222 56,221 63,220 69,218 76,217 83,215 89,213 96,212 103,210 109,208 116,207 123,205 129,204 136,202 143,201 149,199 156,197 163,195 169,194 176,192 183,191 189,190 196,189 203,189 209,188 216,188 223,187 229,186 236,187 242,187 249,188 256,189 262,191 269,193 276,195 282,197 289,200 296,203 302,206 309,210 316,213 322,217 329,221 336,224 342,224 349,221 356,219 362,216 369,214 376,212 382,210 389,208 396,207 402,206 409,205 416,204 422,204 429,204 436,205 442,205 449,205 455,205 462,206 469,206 475,207 482,208 489,209 495,210 502,210 509,211 515,211 522,212 529,212 535,213 542,213 549,214 555,215 562,216 569,217 575,218 582,218 589,219 595,220 602,220 609,221 615,222 622,223 629,223 635,224 "/>
 
alpha and beta
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
-20.0°
 
 
 
-15.0°
 
 
 
-10.0°
 
 
 
-5.0°
 
 
 
0.0°
 
 
 
5.0°
 
 
 
10.0°
 
 
 
15.0°
 
 
 
20.0°
 
 
 
 
0.0°
 
 
 
50.0°
 
 
 
100.0°
 
 
 
150.0°
 
 
 
200.0°
 
 
 
250.0°
 
 
 
300.0°
 
 
 
350.0°
 
 
<polyline fill="none" opacity="1" stroke="#00FF00" stroke-width="1" points="683,194 689,197 696,195 703,205 709,215 716,225 723,229 729,237 736,244 743,252 749,259 756,268 763,277 769,286 776,293 783,300 789,304 796,308 803,314 809,316 816,321 823,325 829,330 836,334 843,337 849,340 856,340 863,341 869,333 876,332 882,326 889,324 896,318 902,312 909,304 916,295 922,285 929,275 936,261 942,245 949,228 956,209 962,200 969,193 976,208 982,244 989,252 996,252 1002,249 1009,237 1016,227 1022,219 1029,213 1036,207 1042,203 1049,201 1056,200 1062,199 1069,192 1076,185 1082,184 1089,180 1095,178 1102,176 1109,173 1115,172 1122,173 1129,172 1135,173 1142,173 1149,175 1155,176 1162,178 1169,182 1175,185 1182,188 1189,193 1195,197 1202,201 1209,203 1215,209 1222,213 1229,217 1235,221 1242,227 1249,229 1255,236 1262,233 1269,237 1275,222 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="683,250 689,258 696,270 703,270 709,270 716,271 723,276 729,277 736,280 743,281 749,282 756,282 763,282 769,282 776,283 783,284 789,286 796,289 803,290 809,292 816,294 823,295 829,294 836,295 843,295 849,295 856,296 863,296 869,302 876,302 882,306 889,305 896,306 902,307 909,307 916,307 922,306 929,304 936,304 942,304 949,304 956,304 962,295 969,281 976,245 982,183 989,158 996,145 1002,136 1009,136 1016,136 1022,135 1029,133 1036,133 1042,133 1049,133 1056,133 1062,134 1069,139 1076,1

In [14]:
solver.particles(8).iterations(30);
let tsr = Array::linspace(1.0,5.0,30);
let cps = Array::from_iter(
    tsr.iter().map(|&tsr| {
        solver.tsr(tsr).solve_with_beta(0.0).c_power()
    })
);
let cps_opt = Array::from_iter(
    tsr.iter().map(|&tsr| {
        solver.tsr(tsr).solve_with_beta_fn(|theta| solution.beta(theta)).c_power()
    })
);
let cps_opt_opt = Array::from_iter(
    tsr.iter().map(|&tsr|{
        solver.tsr(tsr).solve_optimize_beta().c_power()
    })
);

In [15]:
evcxr_figure((640,480), |root| {    
    root.titled("Power coefficient over Tip-Speed-Ratio", ("Arial", 20).into_font())?;
    // Create a chart context with the specified ranges
    let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(1f64..5f64, 0f64..0.6)?;

    // Customize the x-axis labels
    chart.configure_mesh()
        .x_labels(8)
        .y_labels(5)
        .draw()?;

    chart.draw_series(LineSeries::new(
        tsr.iter().cloned().zip(cps.iter().cloned()), 
        &RED
        ))?
        .label("beta = 0°")
        .legend(|(x, y)| PathElement::new(vec![(x, y), (x + 20, y)], &RED));

    chart.draw_series(LineSeries::new(
        tsr.iter().cloned().zip(cps_opt.iter().cloned()),
        &BLUE
        ))?        
        .label("cyclic beta optimized for tsr=2.8")
        .legend(|(x, y)| PathElement::new(vec![(x, y), (x + 20, y)], &BLUE));

    chart.draw_series(LineSeries::new(
        tsr.iter().cloned().zip(cps_opt_opt.iter().cloned()),
        &GREEN
        ))?
        .label("cyclic beta optimized at each tsr")
        .legend(|(x, y)| PathElement::new(vec![(x, y), (x + 20, y)], &GREEN));

    chart.configure_series_labels()
        .border_style(&BLACK)
        .background_style(&WHITE.mix(0.8))
        .draw()?;

    Ok(())
})

Power coefficient over Tip-Speed-Ratio
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.0
 
 
 
0.2
 
 
 
0.4
 
 
 
0.6
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
 
 
 
 
 
beta = 0°
 
 
cyclic beta optimized for tsr=2.8
 
 
cyclic beta optimized at each tsr

In [9]:
solver.tsr(3.25).particles(8);
let c_power: Array1<f64> = Array::from_iter(
    (1..32).map(|iters|{
        solver.iterations(iters).solve_optimize_beta().c_power()
    })
);

evcxr_figure((640,480), |root| {    
    root.titled("Power coefficient with different optimization iterations", ("Arial", 20).into_font())?;
    // Create a chart context with the specified ranges
    let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(1..31, 0f64..0.6)?;

    // Customize the x-axis labels
    chart.configure_mesh()
        .x_labels(8)
        .y_labels(5)
        .draw()?;

    chart.draw_series(LineSeries::new(
        (1..32).zip(c_power.iter().cloned()), 
        &RED
    ))?;
    // chart.draw_series(LineSeries::new(
    //     tsr.iter().cloned().zip(cps_opt.iter().cloned()),
    //     &BLUE
    // ))?;
    Ok(())
})

Power coefficient with different optimization iterations
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.0
 
 
 
0.2
 
 
 
0.4
 
 
 
0.6
 
 
 
 
5
 
 
 
10
 
 
 
15
 
 
 
20
 
 
 
25
 
 
 
30